# Дата цэвэрлэх

In [7]:
# Хэрэгтэй сангуудыг оруулах
import pandas as pd
import re
import os
from datetime import datetime, timedelta

In [8]:
main_path = 'input'
df_18 = pd.read_csv(main_path + '/ads_data_2025-06-18_13-59-54.csv')
df_20 = pd.read_csv(main_path + '/ads_data_2025-06-20_17-44-43.csv')
df_22 = pd.read_csv(main_path + '/ads_data_2025-06-24_17-08-50.csv')

In [9]:
df= pd.concat([df_18, df_20, df_22], ignore_index=True)

### Хэрэггүй багана арилга

In [10]:
df.drop(columns=['page_number','ad_number','phone', 'Төлбөрийн нөхцөл:'], inplace=True)

### Хоосон утгыг устгах

In [11]:
df.isna().sum()

url               0
title             0
location          0
date              0
id                0
price             0
ad_text           0
Талбай:          70
Барилгын явц:    70
clean_coords     15
dtype: int64

In [12]:
df.dropna(subset=['Талбай:', 'Барилгын явц:'], inplace=True )

### Давхацсан утгыг устгах

In [13]:
df[df.duplicated(subset=['id', 'title', 'date'], keep=False)].count()

url              75
title            75
location         75
date             75
id               75
price            75
ad_text          75
Талбай:          75
Барилгын явц:    75
clean_coords     75
dtype: int64

In [14]:
df.drop_duplicates(subset=['id', 'title', 'date'], keep='first',inplace=True)

### Title болон ad_text утгыг шалгаад dic-тэй тохирсон тохиолдол шинэ баганад нэмнэ.

In [15]:
# name.xlsx-ийг ачаалж, dictionary болгох
replace_df = pd.read_excel('D:/Projects/Office/02_data_cleaning/input/name.xlsx', header=1)
replace_df.columns = ['title', 'mytitle']  
replace_dict = dict(zip(replace_df['title'], replace_df['mytitle']))

In [16]:
# Функц: title болон ad_text баганыг шалгаж, тохирсон mytitle-г буцаана
def find_office_name(row):
    texts = []
    if isinstance(row['title'], str):
        texts.append(row['title'].lower())
    if isinstance(row['ad_text'], str):
        texts.append(row['ad_text'].lower())

    for key, value in replace_dict.items():
        key_lower = str(key).lower()
        if any(key_lower in text for text in texts):
            return value
    return None

In [17]:
df['mytitle'] = df.apply(find_office_name, axis=1)
df = df.rename(columns={'mytitle' : 'office_name'})

### Хороо дүүрэг салгах

In [18]:
import util as ut 

In [19]:
 #District name
df['district'] = df['location'].apply(ut.get_district)

In [20]:
#Khoroo
df['khoroo'] = df['location'].apply(ut.get_khoroo)

### Байршил ойролцоогоор хаяглах

In [21]:
# location 
df_loc = pd.read_csv('D:/Projects/Office/02_data_cleaning/input/location.csv')

In [22]:
df = df.merge(df_loc, how='left', on='location')

df['mylocation'].value_counts()
df['mylocation'].isna().sum()


0

In [23]:
df[df['mylocation'].isna()][['location','mylocation']]

,location,mylocation


### Зарим баганы нэрийг өөрчлөх

In [24]:
name_cols = {
           'Title' : 'title',
            'price' : 'price_total',
            'Date'  : 'date',
            'Time'  : 'time',
            'id'    : 'ad_id',
            'loc'   : 'location',
            'message' : 'ad_text',
            'Талбай:' : 'size',
            'Барилгын явц:'          : 'progress_cons',
            'Төлбөрийн нөхцөл:'      : 'payment_terms',
            }

df.rename(columns=name_cols, inplace=True)
df.columns

Index(['url', 'title', 'location', 'date', 'ad_id', 'price_total', 'ad_text',
       'size', 'progress_cons', 'clean_coords', 'office_name', 'district',
       'khoroo', 'mylocation'],
      dtype='object')

### Талбайн болон үнийн мэдээллийг цэвэрлэж тоо болгох

In [25]:
df[['price_total','size']]

,price_total,size
0,482.04 сая ₮,74 м²
1,254.41 сая ₮,39 м²
2,7.3 сая ₮,65 м²
3,1.4 Тэрбум ₮\nүнэ тохирно,210 м²
4,175 сая ₮\n193.95 сая ₮,34 м²
...,...,...
1511,990 сая ₮,220 м²
1512,1.1 Тэрбум ₮,87 м²
1513,685 сая ₮,131 м²
1514,990 сая ₮,135 м²


### Талбайн хэмжээ

In [26]:
df['area'] = df['size'].apply(
    lambda x: float(re.findall(r'\d+[\.\d]*', str(x))[0]) if pd.notnull(x) and re.findall(r'\d+[\.\d]*', str(x)) else None
)
df[['area','size']]
df[['area','size']].dtypes


area    float64
size     object
dtype: object

In [27]:
df = df[df['area'] >= 10 ]

### Оффисын үнэ

Хэрэггүй утгыг арилгаж Float болгох

In [28]:
df['price'] = df['price_total'].apply(lambda x: re.findall(r'\d+[\.\d]*', x)[0]).astype(float)
df[['price','price_total']]
df[['price','price_total']].dtypes

price          float64
price_total     object
dtype: object

Тэр бум гэсэн утгыг тоо болгох

In [29]:
mask = df['price_total'].str.contains('бум', na=False) & (df['price'] < 16)
df.loc[mask, 'price'] = df.loc[mask, 'price'] * 1000

In [30]:
df['price'].describe()
df.sort_values(by='price', ascending=False)

,url,title,location,date,ad_id,price_total,ad_text,size,progress_cons,clean_coords,office_name,district,khoroo,mylocation,area,price
515,https://www.unegui.mn/adv/8411374_ondor-zeregl...,Өндөр зэрэглэлийн оффис таурт,"УБ — Сүхбаатар, Сүхбаатар, Хороо 1",Нийтэлсэн: 2025-06-09 22:06,8411374,10 Тэрбум ₮,"📍Өндөр зэрэглэлийн оффис таурт өндөр давхарт, ...",545 м²,Ашиглалтад орсон,47.912432 106.921754,None,Сүхбаатар,1,Талбай,545.0,10000.0
1026,https://www.unegui.mn/adv/9330971_udirdlgyn-ak...,Удирдлагын академи баруун талд титан таур,"УБ — Хан-Уул, Удирдлагын Академи",Нийтэлсэн: 2025-05-22 09:26,9330971,8.1 Тэрбум ₮,TITAN TOWER-т оффис үйлчилгээний талбай худ...,900 м²,Ашиглалтад ороогүй,47.897313 106.91261,N tower,Хан-Уул,,Зайсан хойно,900.0,8100.0
610,https://www.unegui.mn/adv/9355924_khotyn-aaa-b...,Хотын төвд оффис 850мк талбай,"УБ — Сүхбаатар, Сүхбаатар, Хороо 1",Нийтэлсэн: 2025-06-08 16:12,9355924,7.5 Тэрбум ₮,"📍""Аюуд"" тауэрт давхраар нь оффисийн талбай зар...",850 м²,Ашиглалтад орсон,47.912432 106.921754,None,Сүхбаатар,1,Талбай,850.0,7500.0
57,https://www.unegui.mn/adv/9369577_bzd-khavdar-...,"Бзд, хавдар судлалын буудал, gumuda plaza 960м...","УБ — Баянзүрх, 13-р хороолол",Нийтэлсэн: Өнөөдөр 10:54,9369577,6.72 Тэрбум ₮\nүнэ тохирно,"БЗД, Хавдар судлалын буудал, Gumuda plaza 960м...",960 м²,Ашиглалтад ороогүй,47.913249 106.938278,Gumuda plaza,Баянзүрх,,13-р хороолол,960.0,6720.0
1245,https://www.unegui.mn/adv/9369577_bzd-khavdar-...,"Бзд, хавдар судлалын буудал, gumuda plaza 960м...","УБ — Баянзүрх, 13-р хороолол",Нийтэлсэн: 2025-06-18 10:54,9369577,6.72 Тэрбум ₮\nүнэ тохирно,"БЗД, Хавдар судлалын буудал, Gumuda plaza 960м...",960 м²,Ашиглалтад ороогүй,47.913249 106.938278,Gumuda plaza,Баянзүрх,,13-р хороолол,960.0,6720.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,https://www.unegui.mn/adv/9355976_21-horoolold...,21-r horoolold 141mkv office,"УБ — Сонгинохайрхан, 21-р хороолол",Нийтэлсэн: 2025-06-03 16:33,9355976,2.6 сая ₮\n2.8 сая ₮,21-r horoolold office zarna,141 м²,Ашиглалтад орсон,47.919182 106.814799,None,Сонгинохайрхан,,21-р хороолол,141.0,2.6
1125,https://www.unegui.mn/adv/8734912_dolgoon-nuur...,Сбд дөлгөөн нуурын зүүн талд computer mall офф...,"УБ — Сүхбаатар, Сүхбаатар, Хороо 9",Нийтэлсэн: Өнөөдөр 12:32,8734912,2.5 сая ₮\nүнэ тохирно,✨Дөлгөөн нуурын зүүн талд Computer malld оффис...,434 м²,Ашиглалтад орсон,47.931917 106.912458,None,Сүхбаатар,9,Дөлгөөн нуур,434.0,2.5
699,https://www.unegui.mn/adv/8734912_dolgoon-nuur...,Сбд дөлгөөн нуурын зүүн талд computer mall офф...,"УБ — Сүхбаатар, Сүхбаатар, Хороо 9",Нийтэлсэн: 2025-06-05 12:22,8734912,2.5 сая ₮\nүнэ тохирно,✨Дөлгөөн нуурын зүүн талд Computer malld оффис...,434 м²,Ашиглалтад орсон,47.931917 106.912458,None,Сүхбаатар,9,Дөлгөөн нуур,434.0,2.5
634,https://www.unegui.mn/adv/9151717_3-4-khoroolo...,"Бгд 3, 4 хороололд оффис 120мк талбай","УБ — Баянгол, 3, 4 хороолол",Нийтэлсэн: 2025-06-07 11:32,9151717,2.4 сая ₮\nүнэ тохирно,💎 3.4 хороолол дүүхээ төвийн урд ажил оффис ч...,120 м²,Ашиглалтад орсон,47.924395 106.87749,None,Баянгол,,"3, 4 хороолол",120.0,2.4


1 м.кв-ийн үнийг бодож гаргах

In [31]:
df = df[(df['price'] < 15) | (df['price'] > 50)]  # 15-с 50-ийн хоорондох утгыг хасах

In [32]:
df['price_m2'] = df['price']

In [33]:
mask = df['price'] > 15 #15-с их утгуудыг маск болгож авна. 
df.loc[mask, 'price_m2'] = df.loc[mask, 'price'] / df.loc[mask, 'area'] #15-с их утгыг талбайд нь хувааж 1 м.кв үнийг гаргана
df['price_m2'] = df['price_m2'].round(2) # 10-ны орноор бүхэлчлэв.

1 м.кв-ийн үнээр байгааг нийт үнэ болгох

In [34]:
mask = df['price'] < 15 #15-с их утгуудыг маск болгож авна. 
df.loc[mask, 'price'] = df.loc[mask, 'price_m2'] * df.loc[mask, 'area'] # 1 м.кв-аар байгаа үнийг нийт үнэ болгох
df['price'] = df['price'].round(2) # 10-ны орноор бүхэлчлэв.

In [35]:
df.sort_values(by='price_m2', ascending=False)[['office_name','price', 'area', 'price_m2', 'ad_id']]

,office_name,price,area,price_m2,ad_id
515,None,10000.0,545.0,18.35,8411374
1097,Minister Tower,1940.0,130.0,14.92,8891578
1321,Minister Tower,1940.0,130.0,14.92,8891578
1238,Minister Tower,1743.3,117.0,14.90,8999119
49,Minister Tower,1743.3,117.0,14.90,8999119
...,...,...,...,...,...
96,None,80.0,53.0,1.51,9456693
1277,None,80.0,57.0,1.40,9457079
91,None,80.0,57.0,1.40,9457079
717,None,77.2,285.0,0.27,9415257


### Огноог хувиргах

In [36]:
import pandas as pd
from datetime import datetime, timedelta

# Одоогийн огноог авна (2025-06-20 14:09 гэж авч байна)
today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
yesterday = today - timedelta(days=1)

# Огноог хөрвүүлэх функц
def parse_date(text):
    date_str = text.replace("Нийтэлсэн: ", "").strip()
    if date_str.startswith("Өнөөдөр"):
        time_str = date_str.replace("Өнөөдөр ", "").strip()
        return datetime.strptime(f"{today.strftime('%Y-%m-%d')} {time_str}", "%Y-%m-%d %H:%M")
    elif date_str.startswith("Өчигдөр"):
        time_str = date_str.replace("Өчигдөр ", "").strip()
        return datetime.strptime(f"{yesterday.strftime('%Y-%m-%d')} {time_str}", "%Y-%m-%d %H:%M")
    else:
        return datetime.strptime(date_str, "%Y-%m-%d %H:%M")

# Огноог хөрвүүлэх
df["parsed_date"] = df["date"].apply(parse_date)
# Огноо, цагийг тусад нь баганад оруулах
df["date"] = df["parsed_date"].dt.strftime("%Y-%m-%d")
df["time"] = df["parsed_date"].dt.strftime("%H:%M")

### Хэрэггүй багана устгаж файл хадгалах

Хэрэггүй багана устгах

In [37]:
df.drop(columns=['location', 'price_total', 'size', 'parsed_date'], inplace=True)

Зөв дарааллаар Dataframe үүсгэх

In [38]:
df2 = df[['ad_id', 'district', 'khoroo', 'mylocation','office_name','price', 'area', 'price_m2', 'progress_cons','title', 'ad_text', 'clean_coords','url', 'date', 'time']]

Оффисын нэр, нийт үнэ, талбайн хэмжээ давхацсан заруудыг устгах

In [ ]:
df2.drop_duplicates(subset=['office_name', 'price', 'area'], keep='first', inplace=True)

C:\Users\Rentsen.Ts\AppData\Local\Temp\ipykernel_16996\3988909826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop_duplicates(subset=['office_name', 'price', 'area'], keep='first', inplace=True)


,ad_id,district,khoroo,mylocation,office_name,price,area,price_m2,progress_cons,title,ad_text,clean_coords,url,date,time
0,9299916,Чингэлтэй,,Баянбүрд,Tasgan tower,482.04,74.0,6.51,Ашиглалтад орсон,Тасган товерт оффис 74мк талбай,Хувь лизинг хийх боломжтой\nБизнес эрхлэгчид б...,47.9278806292485 106.90571821241,https://www.unegui.mn/adv/9299916_tasgan-tover...,2025-06-30,09:12
1,9451753,Чингэлтэй,5,5-р сургууль,Tasgan tower,254.41,39.0,6.52,Ашиглалтад орсон,Тасган таурт 39.14мкв оффис талбай,Тасган таурт 39.14мкв оффис талбай худалдана.\...,47.92397 106.902687,https://www.unegui.mn/adv/9451753_tasgan-taurt...,2025-06-30,06:18
2,9457746,Хан-Уул,17,Зайсан хойно,Uniq center,474.50,65.0,7.30,Ашиглалтад орсон,Хан-уул и-мартын урд оффис,Хан-Уул E-mart худалдааны төвийн урд төв зам д...,47.8927222013864 106.923521825545,https://www.unegui.mn/adv/9457746_khan-uul-i-m...,2025-06-29,22:27
3,9455848,Сүхбаатар,,Талбай,None,1400.00,210.0,6.67,Ашиглалтад орсон,Талбайн баруун талд давхар бүхлээрээ оффис 210мкв,Төв талбайн баруун талд Төв шуудангийн ард дав...,47.9188290307491 106.917622814522,https://www.unegui.mn/adv/9455848_talbain-baru...,2025-06-29,14:14
4,9432720,Чингэлтэй,5,5-р сургууль,Tasgan tower,175.00,34.0,5.15,Ашиглалтад орсон,Тасган таурт оффисын 33.44 мкв талбай,"𝗙𝗢𝗥 𝗦𝗔𝗟𝗘\nЧингэлтэй дүүрэг 𝟱-𝗽 𝘅𝗼𝗽𝗼𝗼, Улаанбаа...",47.92397 106.902687,https://www.unegui.mn/adv/9432720_tasgan-taurt...,2025-06-29,12:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,9473048,Баянгол,,Баруун 4 зам,Grand office,1380.00,241.0,5.73,Ашиглалтад орсон,"Баруун 4-н зам, гранд плаза оффис",📍ГРАНД ПЛАЗА-д оффис талбай зарна.\n✔️242.1мкв...,47.916677 106.897889,https://www.unegui.mn/adv/9473048_baruun-4-n-z...,2025-06-22,17:55
1489,9470164,Хан-Уул,,Яармаг,Business tower,5808.00,484.0,12.00,Ашиглалтад орсон,Mcu business tower-т 484 мкв оффис,"📍Яармаг, Хотын захиргааны урд MCU Business Tow...",47.872371 106.814446,https://www.unegui.mn/adv/9470164_mcu-business...,2025-06-21,16:02
1493,9469601,Баянгол,,"3, 4 хороолол",None,316.00,115.0,2.75,Ашиглалтад орсон,"3,4 хороолол гурван билэг их дэлгүүрийн 4 давх...",🏠 3.4 ХОРООЛОЛД ОФФИСООР АЖИЛЛУУЛЖ БОЛОХ 4 ӨРӨ...,47.924395 106.87749,https://www.unegui.mn/adv/9469601_3-4-khoroolo...,2025-06-21,13:46
1496,9296878,Хан-Уул,,Цэнгэлдэх,None,600.00,75.0,8.00,Ашиглалтад орсон,Худ сургалтын чиглэлээр ашиглаж байсан үйлчилг...,"📍Хотын төв, 120мянгат, SS garden-ы баруун талд...",47.903727020255 106.916193816954,https://www.unegui.mn/adv/9296878_surgaltyn-ch...,2025-06-21,12:05


Dataframe хадгалах

In [40]:
# 'result' фолдер үүсгэх
folder_path = "result"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Файлыг 'result' фолдерт хадгалах
file_path = os.path.join(folder_path, "output_06_25.csv")
df2.to_csv(file_path, index=False,  encoding='utf-8-sig')

PermissionError: [Errno 13] Permission denied: 'result\\output_06_25.csv'